In [76]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

In [77]:
driver = webdriver.Chrome()

In [78]:
driver.get("https://www.playbill.com/production/gross?production=00000150-aea5-d936-a7fd-eef572240001")

In [79]:
col0 = driver.find_elements(by = By.CLASS_NAME, value = "col-0")
col1 = driver.find_elements(by = By.CLASS_NAME, value = "col-3")
col2 = driver.find_elements(by = By.CLASS_NAME, value = "col-4")
col3 = driver.find_elements(by = By.CLASS_NAME, value = "col-6")

In [80]:
def check_elements(iter1):
    for e in iter1:
        print(e.text)
    print(len(iter1))

In [63]:
# # For checking the structuring of the elements which have been scraped
# check_elements(col0)
# check_elements(col1[1:])
# check_elements(col2[1:])
# check_elements(col3[1:])

Conclusion: 
1. col0 starts with column name and continues with column values. Can be used as is, except that column header should be sliced out.
2. col1 starts with a column name, which is, however, a wrong label for the data, the html source for the website has repeated a class "col-1" twice. Thus, the column is actually "Avg Ticket - Top Ticket" and the text under it should be split into 2 different columns.
3. col2 is similar to col1, and the column name is also wrong, it should be - "Seats sold - Seats in theatre." The second column value is seen to remain mostly constant and might be truncated because it doesn't provide any additional insight.

Added Edit:
4. col3 is the % of seats filled (% cap), however, is wrongly labelled as perfs.

In [81]:
i = 1
dfcol1 = [] # Avg Ticket
dfcol2 = [] # Top Ticket
dfcol3 = [] # Seats sold
dfcol4 = [] # Seats in theatre
for x, y in zip(col1[1:], col2[1:]):
    l1, l2 = x.text.split('\n'), y.text.split('\n')
    dfcol1.append(l1[0])
    dfcol2.append(l1[1])
    dfcol3.append(l2[0])
    dfcol4.append(l2[1])

In [87]:
next_page = driver.find_element(by = By.CLASS_NAME, value = "bsp-pagination-navigate-next")

In [88]:
next_page.click()

In [89]:
col0 = driver.find_elements(by = By.CLASS_NAME, value = "col-0")
col1 = driver.find_elements(by = By.CLASS_NAME, value = "col-3")
col2 = driver.find_elements(by = By.CLASS_NAME, value = "col-4")
col3 = driver.find_elements(by = By.CLASS_NAME, value = "col-6")

In [ ]:
# For checking the structuring of the elements which have been scraped
check_elements(col0)
check_elements(col1[1:])
check_elements(col2[1:])
check_elements(col3[1:])

In [67]:
driver.quit()

In [93]:
def collect_data(df_list):
    dfcol0 = df_list[0] # Week Ending
    dfcol1 = df_list[1] # Avg Ticket
    dfcol2 = df_list[2] # Top Ticket
    dfcol3 = df_list[3] # Seats sold
    dfcol4 = df_list[4] # Seats in the Theatre
    dfcol5 = df_list[5] # % Cap

    driver = webdriver.Chrome()
    data_link = "https://www.playbill.com/production/gross?production=00000150-aea5-d936-a7fd-eef572240001"
    driver.get(data_link)

    while (True):
        col0 = driver.find_elements(by = By.CLASS_NAME, value = "col-0")
        col1 = driver.find_elements(by = By.CLASS_NAME, value = "col-3")
        col2 = driver.find_elements(by = By.CLASS_NAME, value = "col-4")
        col3 = driver.find_elements(by = By.CLASS_NAME, value = "col-6")

        for e0, e1, e2, e3 in zip(col0[1:], col1[1:], col2[1:], col3[1:]):
            l1 = e1.text.split('\n')
            l2 = e2.text.split('\n')
            dfcol0.append(e0.text)
            dfcol1.append(l1[0])
            # Top ticket values don't exist for some pages
            if (len(l1) == 2):
                dfcol2.append(l1[1])
            else:
                dfcol2.append(np.NaN)
            dfcol3.append(l2[0])
            dfcol4.append(l2[1])
            dfcol5.append(e3.text)

        try:
            next_page = driver.find_element(by = By.CLASS_NAME, value = "bsp-pagination-navigate-next")
        except NoSuchElementException:
            print("Finished collecting data!")
            driver.quit()
            break
        except Exception as e:
            print(f"An exception {e} occurred, ceasing process.")
            driver.quit()
            break
        next_page.click()

    return df_list


In [94]:
df_list = [[], [], [], [], [], []]
collect_data(df_list)

NameError: name 'null' is not defined

In [ ]:
print("Week Endings:\n")
print(df_list[0][10:17])
print("Avg Tickets:\n")
print(df_list[1][10:17])
print("Top Tickets:\n")
print(df_list[2][10:17])
print("Seats sold:\n")
print(df_list[3][10:17])
print("Seats in theatre:\n")
print(df_list[4][10:17])
print("% Cap:\n")
print(df_list[5][10:17])